# Why Apache Arrow?

> **Level:** Beginner  
> **Spec:** [Arrow Columnar Format Introduction](https://arrow.apache.org/docs/format/Columnar.html)  
> **PyArrow docs:** [Getting Started](https://arrow.apache.org/docs/python/getstarted.html)

## What you will learn

1. Why columnar storage outperforms row storage for analytics
2. Arrow's core guarantees: shared memory, zero-copy, ABI stability
3. Vocabulary: `Array`, `Buffer`, `Schema`, `RecordBatch`, `Table`

In [ ]:
import time
import pyarrow as pa
import pyarrow.compute as pc
import numpy as np

---
## 1. Row vs Columnar Storage

> **Further reading:** [Why columnar?](https://arrow.apache.org/docs/format/Columnar.html#physical-memory-layout)

Imagine a table with 1 million student records: `(name, score, grade)`.  
An analytics query that computes **sum(score)** needs only the `score` column.  

| Storage | Data read | Cache behaviour |
|---------|-----------|----------------|
| Row (CSV / SQL heap) | Entire row (name + score + grade) | Poor: many cache misses |
| Columnar (Arrow) | Score buffer only | Excellent: sequential scan |

In [2]:
N = 1_000_000
rng = np.random.default_rng(42)
scores_np = rng.integers(0, 100, size=N)

# Row-oriented: list of dicts (Python objects, no contiguous memory)
rows = [{"name": f"student_{i}", "score": int(scores_np[i]), "grade": "A"} for i in range(N)]

# Column-oriented options
scores_list = scores_np.tolist()          # Python list
arrow_scores = pa.array(scores_np, type=pa.int32())  # Arrow array

# Benchmark: sum(score)
t0 = time.perf_counter()
row_sum = sum(r["score"] for r in rows)
t_rows = (time.perf_counter() - t0) * 1000

t0 = time.perf_counter()
list_sum = sum(scores_list)
t_list = (time.perf_counter() - t0) * 1000

t0 = time.perf_counter()
arrow_sum = pc.sum(arrow_scores)
t_arrow = (time.perf_counter() - t0) * 1000

print(f"Sum (all agree): {row_sum == list_sum == arrow_sum.as_py()}")
print(f"\nRow iteration  : {t_rows:7.1f} ms")
print(f"Python list    : {t_list:7.1f} ms")
print(f"Arrow compute  : {t_arrow:7.1f} ms")
print(f"\nArrow speedup vs rows : {t_rows/t_arrow:.0f}x")

Sum (all agree): True

Row iteration  :    29.1 ms
Python list    :     4.8 ms
Arrow compute  :     0.4 ms

Arrow speedup vs rows : 72x


---
## 2. Arrow's Four Guarantees

> **Spec:** [Goals](https://arrow.apache.org/docs/format/Columnar.html#goals)

| Guarantee | Meaning |
|-----------|---------|
| **Contiguous buffers** | Each column is a flat byte buffer: CPU-friendly, SIMD-ready |
| **Shared memory** | Multiple processes can map the same buffer (no copy) |
| **Language-agnostic** | Same binary layout in Python, Rust, Java, C++, Go, R … |
| **Null support** | A validity *bitmap* encodes missing values without NaN hacks |

In [3]:
# Arrow vocabulary:

# Buffer: raw bytes
buf = pa.py_buffer(b"\x01\x00\x03\x00\x04\x00")  # 3 x int16
print("Buffer bytes:", buf.to_pybytes().hex())

# Array: typed view of buffers
arr = pa.array([1, None, 3, 4], type=pa.int32())
print("\nArray:", arr)
print("Type:", arr.type)
print("Null count:", arr.null_count)
print("Buffer count:", len(arr.buffers()))

# Schema: named fields with types
schema = pa.schema([
    pa.field("name",  pa.utf8()),
    pa.field("score", pa.int32()),
    pa.field("grade", pa.utf8()),
])
print("\nSchema:", schema)

# RecordBatch: schema + equal-length column arrays
batch = pa.record_batch({
    "name":  ["Alice", "Bob", "Carol"],
    "score": [95, 82, 91],
    "grade": ["A", "B", "A"],
}, schema=schema)
print("\nRecordBatch:\n", batch.to_pandas())

# Table: collection of RecordBatches (possibly chunked)
table = pa.table({"score": pa.chunked_array([pa.array([95, 82]), pa.array([91, 77])])})
print("\nTable num_chunks:", table.column(0).num_chunks)

Buffer bytes: 010003000400

Array: [
  1,
  null,
  3,
  4
]
Type: int32
Null count: 1
Buffer count: 2

Schema: name: string
score: int32
grade: string


TypeError: Unsupported dataframe type, got: <class 'pyarrow.lib.RecordBatch'>

---
## Summary

- Columnar layout enables sequential memory access → cache-friendly → fast SIMD aggregation
- Arrow arrays are backed by contiguous `Buffer` objects with a validity bitmap for nulls
- `RecordBatch` = schema + equal-length columns; `Table` = chunked collection of batches
- Arrow unifies in-memory representation across languages, one buffer, many readers

**Next ⏭️** [Inspect the raw bytes of Arrow buffers](02_arrays_buffers_nulls.ipynb)